In [1]:
import os
from pathlib import Path

import numpy as np
from preprocessing import preprocess
from collection import s3_download_file

In [2]:
# Install the ONNX runtime to run inference
%pip install onnxruntime==1.18.1
import onnxruntime  # noqa


[notice] A new release of pip available: 22.2.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Define the accuracy threshold required on the test set
EVAL_ACCURACY_THRESHOLD = os.getenv("eval_accuracy_threshold", 0.8)

In [4]:
# Download an unzip the dataset
S3_DATA_FILE = os.getenv("s3_data_file", "kagglecatsanddogs_5340.zip")
DOWNLOAD_PATH = Path(".cache/data.zip")

if not DOWNLOAD_PATH.is_file():
    s3_download_file(S3_DATA_FILE, DOWNLOAD_PATH)
    !unzip -n -q .cache/data.zip -d .cache

In [5]:
# Directory path of your dataset
data_dir = '.cache/PetImages'

# Preprocess the data, but we only need the test data loader for evaluation
_, _, test_loader, _ = preprocess(data_dir)

Deleted 0 corrupted images.


In [6]:
session = onnxruntime.InferenceSession("model.onnx")

correct = 0

for inputs, labels in test_loader:
    # Convert PyTorch tensor to a NumPy array, expected by the ONNX model
    onnx_inputs = {"input": inputs.numpy().astype(np.float32)}
    onnx_output_names = ["output"]

    result = session.run(onnx_output_names, onnx_inputs)
    outputs = result[0]
    preds = np.argmax(outputs, axis=1)
    correct += np.sum(preds == labels.numpy())

test_accuracy = correct / len(test_loader.dataset)

test_accuracy

0.8763884933067502

In [7]:
assert test_accuracy >= EVAL_ACCURACY_THRESHOLD, (
    f"The model is not accurate enough ({test_accuracy}). "
    "Deployment will be cancelled."
)